Performs landmark localisation on the 300vw testset.

It works in batch mode, i.e. run the landmark localisation for

all the methods defined. 

_______________________________________________________________



It expects the following inputs:

   a) The trained landmark localisation models,

   b) The bounding boxes of the trackers/detectors exported,

   c) The frames.


_______________________________________________________________

The structure is assumed the same as in all notebooks, i.e. as 

indicated below.



It is highly recommended to use zero-padding in the frame names and

the respective landmark files, because otherwise it depends on the OS

to interpret the sequence of frames.

_______________________________________________________________

Developed on menpo.0.6 version.

Updated for the latest menpo.0.7.5 version, September 2016. 


```
(expected folder structure)

path_base_testset
└───category1
    │
    └─── frames
            │
            └─── [name_of_clip] (e.g. 541)
                    │ [frame_name].[extension] (e.g. 000001.png)
                    │ [frame_name].[extension] (e.g. 000002.png)
                    │ ...
            │ ...
    │
    └─── gt_landmarks  (not required for executing the code)
            │
            └─── [name_of_clip]
                    │ [file_name].[extension] (e.g. 000001.pts)
                    │ ...
            │ ...
    │
    └─── [detector/tracker]_[method_name] (e.g. detector_opencv, tracker_cmt)
            │
            └─── [name_of_clip]
                    │ [file_name].[extension] (e.g. 000001.pts)
                    │ ...
            │ ...
 └───category2
    │ ...
```

In [ ]:
from __future__ import division
from os.path import isdir, join, isfile, sep
from os import listdir 
from functools import partial
from warnings import warn
import numpy as np
import logging
from pathlib import Path
from workerbee import exhaust_all_files_randomly

# menpo packages imports
import menpo.io as mio
from menpodetect.dlib.conversion import pointgraph_to_rect
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup
from menpo.transform import Scale, Translation

try:
    from research_pyutils import mkdir_p, rm_if_exists, execution_stats
    from research_pyutils.menpo_related import compute_overlap
except ImportError:
    m1 = ('The import failed, please check that you have the\n'
          'package research_pyutils from here: \n'
          'https://github.com/grigorisg9gr/pyutils \n')
    raise ImportError(m1)

In [ ]:
# auxiliary, print stats, no functional purpose
execution_stats()

## Base paths 

In [ ]:
# base path where the frames and landmarks exist/will be saved into.
path_base = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/300vw_testset/'
assert(isdir(path_base))

# base path for the trained landmark localisation models.
path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

## All functions 

In [ ]:
def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

In [ ]:
def _get_path(p0, name):
    # Join the path and ensure it exists.
    assert(isdir(p0))
    p1 = join(p0, name, '')
    if not isdir(p1):
        mkdir_p(p1)
    return p1


def return_scale(pt, target_sz):
    # find the current range of the bb (pt) and the scaling required 
    # to reach the target_sz. Used for the regression in 
    # detector's AAM only. 
    target_sz = np.array(target_sz) * 1.0
    range0 = np.max(pt, axis=0) - np.min(pt, axis=0)
    assert(np.min(range0) > 0)
    scale = target_sz / range0
    return scale


def aam_regression(im, g):
    # using global regr1, reg_f.
    # The function resizes the bb based on the regression 
    # training, that is to  scale/translate the detectors' 
    # bb towards the gt bb.  It initially resizes the image, 
    # extracts the feats, acquires  the prediction and then 
    # applies it on the detectors' bb.
    
    # Resizes the images based on the target_sz, extracts the 
    # feats in the resized bb, fit the regression, gets the 
    # prediction and applies the transform to the image.
    target_sz = reg_f['target_size']
    feats = reg_f['feats']
    
    sc1 = return_scale(im.landmarks[g].lms.points, target_sz)
    im2 = im.resize(sc1 * im.shape)

    pt = im2.landmarks[g].lms.points
    min_idx = np.round(np.min(pt, axis=0))
    # often the size of the bb is 1 pixel off, so we specifically crop 
    #it to be in the target_sz.
    im3 = feats(im2.crop(min_idx, min_idx + target_sz))
    ft = np.reshape(im3.pixels, -1)
    # predict from regression method
    ft = ft.reshape(1, -1)  # avoid annoying warning of sklearn
    pred = regr1.predict(ft)
    
    # get the reshaped bb, based on the prediction of the regressor.
    # WARNING: The scaling as defined in the training is a pure scaling of the 
    # width, height, while with the transform in menpo, the bounding box moves
    # towards [0,0], so we account for this change and add it to the translation.
    scale = Scale(pred[0, 0 : 2])
    centre = im.landmarks[g].lms.centre()
    pcloud = scale.apply(im.landmarks[g].lms)
    im.landmarks['pred'] = pcloud
    diff_centre = centre - im.landmarks['pred'].lms.centre()
    
    transl = Translation(pred[0, 2 : 4] + diff_centre)
    pcloud = transl.apply(im.landmarks['pred'].lms)
    # the new bb is saved as pred landmark in the original image
    im.landmarks['pred'] = pcloud
    
    return pred
    
    
def process_frame(p_fr):
    """
    Main processing function per frame. 
    It loads the frame, applies landmark localisation
    technique and exports the landmark file.
    
    The function predicts several corner cases that emerged
    during fitting different methods, thus the code was 
    adapted to fit those, thus it is greater in extent than
    a simple fitting. 
    
    An additional complication factor is the workerbee for
    running the fitting in a batch mode (cluster). 
    Specifically, several variables are globals, due to the
    current workerbee version that accepts only p_fr as the
    variable to pass to the calling function.
    
    Furthermore, this workerbee version decides on the next
    task based on the written filenames, thus for every 
    frame processed, there is a dummy file written in a 
    different path, even if the landmark localisation 
    technique is not applied. 
    
    Globals used: p_ln_out_0, model, p_condor_dummy_0, 
                  p_bb_out_0, min_sz, method_landm_loc,
                  fold_out.
    
    """
    try:
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
        # # # # # #   Load the image and search for detections.   # # # # # #
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
        im = mio.import_image(p_fr, landmark_resolver=None)
        if im.n_channels == 3:
            im = im.as_greyscale()
        p_bb_out = _get_path(p_bb_out_0, im.path.parent.name) + im.path.stem
        if isfile(p_bb_out + '.pts'):  # allow the _0.pts extension
            p_bb_out += '.pts'
        else:
            p_bb_out += '_0.pts'

        if isfile(p_bb_out):
            # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
            # # # # #  Load the detection and perform sanity checks.  # # # # # #
            # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
            ln = mio.import_landmark_file(p_bb_out)
            if ln.lms.n_points == 0:
                print('The ln {} has 0 values in the ln.'.format(ln.path.stem))
                # create dummy file for workerbee
                p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
                open(p_cond + im.path.stem + '.pts', 'a').close() 
                return  
            assert(ln.lms.n_points == 4)
            
            # catch the case of nan or overflowed values and don't try to fit those.
            if ln.lms.has_nan_values() or np.any(ln.lms.points > 1e6):
                print('The ln {} has nan or overflowed values.'.format(ln.path.stem))
                # create dummy file for workerbee
                p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
                open(p_cond + im.path.stem + '.pts', 'a').close() 
                return                

            p_out = _get_path(p_ln_out_0, im.path.parent.name) + im.path.stem + '.pts'
            # ensure that the bb is inside the bounds
            try:
                ln = ln.lms.constrain_to_bounds(im.bounds())
            except:
                # The code below is for menpo versions before 0.7.
                # This will be removed in future version though.
                im.landmarks['bb'] = ln
                im.constrain_landmarks_to_bounds()
                ln = im.landmarks['bb']
            
            # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
            # # # # # #  Perform landmark localisation for a method.  # # # # # #
            # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
            if method_landm_loc == 'dlibERT':
                im_pili = np.array(im.as_PILImage())
                det_frame = model(im_pili, pointgraph_to_rect(ln.lms))
                init_pc = detection_to_pointgraph(det_frame)
                mio.export_landmark_file(LandmarkGroup.init_with_all_label(init_pc), 
                                         p_out, overwrite=True)
            else:
                # The following 'if' hack to load regression model for AAM.
                if ('aam' in fold_out) and ('detection' in fold_out):
                    pred = aam_regression(im, 'bb')
                    ov1 = compute_overlap(im.landmarks['pred'].lms.points, 
                                          ln.lms.points)
                    # detectors are not too off in their prediction, so require
                    # some minimum overlap to replace with the regression's bb.
                    if ov1 > 0.45 :
                        ln = im.landmarks['pred']
                        ln = ln.lms.constrain_to_bounds(im.bounds())
                        
                # request a minimum size of bb, otherwise menpofit might crash.
                bp = ln.lms.points
                if np.all(np.max(bp, 0) - np.min(bp, 0) >= min_sz): 
                    ft = model.fit_from_bb(im, ln.lms)
                    im.landmarks['gg'] = ft.final_shape
                    mio.export_landmark_file(im.landmarks['gg'], p_out, overwrite=True)
                    print(' successfully fitted')  # helps in condor

        
        # create dummy file for workerbee
        p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
        open(p_cond + im.path.stem + '.pts', 'a').close() 
    except Exception as e:
        # We catch any type of exception here, in order to allow the 
        # algorithm to continue the execution with the rest landmarks.
        # Can be off for debugging or experimentation. 
        print(e)
        if 'p_out' in locals():
            print("The input '{}' failed with p_ln '{}'.".format(p_fr, p_out))
        logging.exception("The input '{}' failed.".format(p_fr))
        if ('aam' in fold_out) and ('detection' in fold_out):
            # hack to load regression model 
            try:
                print(e)
                # create dummy file for workerbee
                p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
                open(p_cond + im.path.stem + '.pts', 'a').close() 
            except:
                pass
        else:
            raise ValueError()

## Selection of methods 

In [ ]:
# minimum size of bb we 'allow'. Less than that, there is no 
# fitting in SDM/AAM.
min_sz = np.array([10, 10])
# landmark localisation method for which all the bounding
# boxes will be fit. Ensure that the respective model exists.
method_landm_loc = 'aam'


# Below are all the methods for which the landmarks will be fit. 
# They are separated into detection and tracking methods due 
# to the different models required, as well as for the different
# pattern of their bb paths.
detections_methods = ['dlib', 'opencv', 'ffld2', 'ramanan']
tracking_methods = ['corr', 'fct', 'rpt', 'lrst', 'spot', 'tld', 'srdcf', 
                   'kcf', 'cmt', 'mil', 'struck', 'ivt', 'df',
                   'staple', 'lct', 'meem', 'sir_pf', 'camshift']
tracking_methods = ['superpixel']
detections_methods = []
# auxiliary lambda function used in the following lines
nn = lambda n, i: n + i + '_' + method_landm_loc
# detectors' pool
fold_out_pool_0 = [nn('detection_', i) for i in detections_methods]
f0 = ['detector_' + i for i in detections_methods]
# trackers' pool
fold_out_pool_1 = [nn('tracking_', i) for i in tracking_methods]
f1 = ['tracker_' + i  for i in tracking_methods]
# combine the two lists
f_o_pool = fold_out_pool_0 + fold_out_pool_1
f_d_o_pool = f0 + f1
names = detections_methods + tracking_methods
assert(len(f_d_o_pool) == len(f_o_pool))

## Main processing 

In [ ]:
cats = listdir(path_base)

for f0 in range(len(f_o_pool)):
    # iterate over all the tracking/detection methods.
    fold_out = f_o_pool[f0]
    fold_det_out = f_d_o_pool[f0]
    print('method: {}, detector: {}'.format(fold_out, fold_det_out))

    # auxiliary for selecting the ll model.
    if fold_det_out[:7] == 'tracker':
        model_bb = 'gt_bb'
    elif fold_det_out[:8] == 'detector':
        model_bb = names[f0]
    else:
        raise ValueError('Not a valid option')
    
    # load prediction model
    if method_landm_loc == 'dlibERT':
        str1 = '{}modelln_{}_{}.model'.format(path_pickles, model_bb, method_landm_loc)
        path_shape_pred = str1
        model = shape_predictor(path_shape_pred)
    else:
        str1 = '{}modelln_{}_{}.pkl'.format(path_pickles, model_bb, method_landm_loc)
        path_pkl = str1
        model = mio.import_pickle(path_pkl)
        
    if ('aam' in fold_out) and ('detection' in fold_out):  # hack for detector's AAM.
        p_regr = join(path_pickles, 'regressor_' + fold_det_out + '.pkl')
        assert(isfile(p_regr))
        reg_f = mio.import_pickle(p_regr)
        regr1 = reg_f['regr']
        
    if 'aam' in fold_out:
        # for compatibility with >=menpo.v0.7.2 and the respective menpofit
        model._reference_shape = model.aam.reference_shape
        model._scales = model.aam.scales
        model._holistic_features = model.aam.holistic_features
    elif 'sdm' in fold_out:
        try:
            # grigoris, 6/2016: This is a hack for ensuring compatibility with the 
            # latest menpofit. That is converting the earlier trained model to the
            # latest release. This should not be required in case you train a new
            # model with the current menpofit (and use it in the same version).
            model._reference_shape = model.__dict__['reference_shape']
            model._scales = model.__dict__['scales']
            model._holistic_features = model.__dict__['holistic_features']
        except KeyError:
            # That's fine, maybe the model was trained with the latest menpofit.
            pass
        
    # for each category in the testset, run landmark loc method
    for cat in cats:
        if not cat[:8] == 'category' or not isdir(path_base + cat):
            warn('Unknown content in path {} (folder: {}).'.format(path_base, cat))
        print(cat)
        # join or create the paths
        p_cat = join(path_base, cat, '')
        # path of the frames:
        p_fr = join(p_cat, 'frames', '')
        # path where the ll files will be exported to.
        p_ln_out_0 = mkdir_p(join(p_cat, fold_out, ''))
        # path where the bb's of each frame (per clip) are located.
        p_bb_out_0 = mkdir_p(join(p_cat, fold_det_out, ''))
        # a dummy path for the workerbee files.
        p_condor_dummy_0 = mkdir_p(join(p_cat, 'condor_tmp', 
                                        'condor_dummy_' + fold_out, ''))
        assert(isdir(p_fr))

        for c in sorted(listdir(p_fr)):   # for each clip
            output_dir = Path(mkdir_p(p_condor_dummy_0 + c + sep))
            done = lambda: output_dir.glob('*.pts')
            im_paths = lambda: mio.image_paths(p_fr + c + sep + '*')
            exhaust_all_files_randomly(im_paths, done, process_frame, verbose=True)
        # uncomment below, only if NOT called in condor.
    #     rm_if_exists(p_condor_dummy_0)
    del model


In [ ]:
print()
execution_stats()